In [1]:
import pandas as pd
import numpy as np
import dash
from dash import Dash, dcc, html, Input, Output, callback
from dash.dependencies import Input, Output
from sqlalchemy import create_engine
import psycopg2
import os
import plotly.figure_factory as ff
import plotly.express as px
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.colors as mcolors
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Polygon
import geoplot as gplt
from geodatasets import get_path
from shapely import wkt
import json
import dash_daq as daq
import pyproj
from plotly import graph_objects as go
import dash_bootstrap_components as dbc

In [2]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'tides'
))
external_stylesheets=[dbc.themes.DARKLY]

# DashBoard

In [3]:
colors = {
    'background': '#171B25',
    'text': '#A0A6B8'
}
def state_names():
    myquery=f'''
    SELECT state as value, name as label
    FROM states_id s
    ORDER BY name
    '''
    return pd.read_sql_query(myquery, con = engine).to_dict('records')

state_options = state_names()


myquery= '''
    SELECT *
    FROM sites_id
    '''
df =gpd.GeoDataFrame.from_postgis(myquery, con = engine, geom_col='geometry') 
symbol_map = {
    'Dune': 'circle',
    'Cliff': 'diamond'}
df['symbol'] = df['feature'].map(symbol_map)
fig = px.scatter_mapbox(df, 
                        lat=df.geometry.y,
                        lon=df.geometry.x,
                        mapbox_style="carto-positron",
                        color = 'feature',
                        hover_name='feature',
                        labels={'beachloc': 'Beach Location'}
                       )
fig.update_traces(marker=dict(size=5)) 
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
sites = fig


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    # Row for the top section
    html.Div([
        # Left side
        html.Div(style={'width': '20%', 'display': 'inline-block'}),

        
        html.Div([
            # Tab 1
            dbc.Tabs(
                id='tabs',
                children=[
                    
                    # Tab 1
                    
                    dbc.Tab(
                        label = "Beach Renourishment by State",
                        children = [
                            html.Div(
                                [
                                    dcc.Markdown('Explore coastal conservation practices within the US over time by state. Interact with the slider to see how beach nourishment volumes have changed over the years.'),
                                ],
                                style={'paddingTop': '20px', 'paddlingRight': '30px'}  
                                ),
                            html.Div(
                                dcc.Graph(id = "map"), style = {'paddingTop': '20px','paddingBottom': '20px', 'paddingRight': '30px'},
                            ),
                            html.Div(
                                dcc.Slider(id='year',
                                         min = 1924,
                                         max = 2022,
                                         step = 2,
                                         marks={year: str(year) for year in range(1926, 2023, 4)}
                                         ), style = {'paddingTop': '20px', 'paddingBottom': '20px'},
                            ),
                            html.Div(
                                dcc.Graph(id="top10"), style = {'paddingTop': '20px', 'paddingBottom': '20px', 'paddingRight': '30px'}
                                    )
                                ]),
                    # Tab 2
                    
                    dbc.Tab(
                        label = "Beach Renourishment by County",
                        children = [
                            html.Div(
                                dcc.Markdown('Explore coastal conservation practices within the US over time by county. Interact with the slider to see how beach nourishment volumes have changed over the years.'),style={'paddingTop': '20px', 'paddlingLeft': '30px'}  
                            ), 
                            html.Div(
                                dcc.Graph(id = 'county'), style = {'paddingTop': '20px', 'paddingRight': '30px', 'paddingBottom': '20px'},
                                    ),
                            html.Div(
                                dcc.Slider(id='year2',
                                         min = 1924,
                                         max = 2022,
                                         step = 2,
                                         marks={year: str(year) for year in range(1926, 2023, 4)}
                                         ), style = {'paddingTop': '20px', 'paddingBottom': '20px', 'paddingRight': '30px'}
                                    ),
                            html.Div(
                                dcc.Graph(id="top10county"), style = {'paddingTop': '20px', 'paddingBottom': '20px', 'paddingRight': '30px'}
                            )
                                ]
                            ),
                    
                    # Tab 3
                    
                    dbc.Tab(
                        label = "Sand Dunes & Cliffs",
                        children = [
                            html.Div(
                                [
                                    dcc.Markdown('Explore where sand dunes and cliffs are located in the US. Currently, this selection only displays these features in select areas.'),
                                ],
                                style = {'paddingTop': '20px'}
                            ),
                            html.Div(
                                dcc.Graph(figure = sites)
                            )
                        ], style = {'paddingRight': '30px'},
                    ),

                    # Tab 4

                    dbc.Tab(
                        label = "Artificial Reefs",
                        children = [
                            html.Div(
                                [
                                dcc.Markdown('Explore the development of artificial reefs. Interact with the slider to see how artificial reefs have developed over the years.'),
                                ],
                                style={'paddingTop': '20px'}
                                ),
                            html.Div( 
                                dcc.Graph(id = "reefs")
                                    ),
                            html.Div(
                                dcc.Slider(id='year1',
                                         min = 2001,
                                         max = 2022,
                                         step = 2,
                                         marks={year: str(year) for year in range(2001, 2023, 4)}
                                         ), 
                                        style = {'paddingTop': '20px', 'paddingBottom': '20px'}
                            )
                        ], style = {'paddingRight': '30px'}
                    )

                ],
            ),
        ], style={'width': '65%', 'display': 'inline-block', 'float': 'right'}),
    ]),

    html.Div([
    dcc.Markdown('''
        Welcome to the 
        
        ## Coast Line Conservation Tracker

        This platform is dedicated to advancing the awareness of coastal conservation practices in the United States over the past century.

        The data from this platform is sourced from the following:
        * [United States Census Bureau](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)
        * [American Shore and Beach Preservation Association](https://asbpa.org/national-beach-nourishment-database/)
        * [National Oceanic and Atmospheric Administration / United States Geological Survey](https://coastal.er.usgs.gov/hurricanes/research/twlviewer/)
    ''', 
    style={'textAlign': 'left', 'color': colors['text']}),
], style={'width': '30%', 'display': 'inline-block', 'paddingLeft': '30px'})
])

@app.callback([Output(component_id = 'map', component_property = 'figure')],
             [Input(component_id = 'year', component_property = 'value')])
def year(b):
    if b is None:
        b = 2022
    myquery = f'''
        SELECT s.state, s.name, s.state_id, s.geometry, SUM(sv.volume_cy) as total_volume
        FROM states_id s
        LEFT JOIN states_volume sv ON sv.state = s.state
        WHERE sv.year <= {b}
        GROUP BY s.state, s.name, s.state_id, s.geometry
        '''
    df = gpd.GeoDataFrame.from_postgis(myquery, con=engine, geom_col='geometry')
    fig = px.choropleth_mapbox(df, 
                    geojson=df.geometry,
                    locations=df.index,
                    color="total_volume",
                    color_continuous_scale="Mint",
                    hover_name='name',
                    mapbox_style="carto-positron",
                    labels={'total_volume': 'Total Volume (cy)'},
                    zoom=1
                      )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return [fig]

@app.callback([Output(component_id = 'top10', component_property = 'figure')],
             [Input(component_id = 'year', component_property = 'value')])

def top10(b):
    if b is None:
        b = 2022
    myquery = f'''
    SELECT s.state, s.name, s.state_id, s.geometry, SUM(sv.volume_cy) as total_volume
    FROM states_id s
    LEFT JOIN states_volume sv ON sv.state = s.state
    WHERE sv.year <= {b}
    GROUP BY s.state, s.name, s.state_id, s.geometry
    '''
    df = pd.read_sql_query(myquery, con=engine)
    df['total_volume'] = df['total_volume'].astype(int)

    # Create a separate column for formatted total_volume
    df['formatted_total_volume'] = df['total_volume'].apply(lambda x: f"{x:,}")

    df_top10 = df.sort_values(by='total_volume', ascending=False).head(10)
    
    fig = px.bar(df_top10, x='state', y='total_volume',
             labels={'total_volume': 'Total Volume (cy)'},
             color='total_volume',  
             color_continuous_scale=px.colors.sequential.Mint
                )
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>Total Volume: %{y:,}")
    
    fig.update_layout(
        plot_bgcolor='white',  
        paper_bgcolor='white',  
        showlegend=False,
        xaxis=dict(showgrid=False), 
        yaxis=dict(showgrid=False),
    )
    return [fig]

@app.callback([Output(component_id = 'county', component_property = 'figure')],
             [Input(component_id = 'year2', component_property = 'value')])

def county(b):
    if b is None:
        b = 2022
    myquery = f'''
        SELECT c.GEO_ID, c.geometry, c.state, c.name, b.year, SUM(b.volume_cy) AS total_volume, c.countyname
        FROM counties_id c
        JOIN beaches_id b ON b.GEO_ID = c.GEO_ID
        WHERE b.year <= {b}
        GROUP BY c.GEO_ID, c.geometry, c.state, c.name, b.year, c.countyname
        '''
    df = gpd.GeoDataFrame.from_postgis(myquery, con=engine, geom_col='geometry')
    
    # Create choropleth map
    fig = px.choropleth_mapbox(df, 
                               geojson=df.geometry,
                               locations=df.index,
                               color='total_volume',
                               color_continuous_scale="Mint",
                               hover_name='countyname',
                               mapbox_style="carto-positron",
                               zoom=3,
                               center={"lat": 37.0902, "lon": -95.7129},
                               labels={'total_volume': 'Total Volume (cy)'})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return [fig]

@app.callback([Output(component_id = 'top10county', component_property = 'figure')],
             [Input(component_id = 'year2', component_property = 'value')])

def countybar(b):
    if b is None: 
        b = 2022
    myquery = f'''
        SELECT c.GEO_ID, c.state, c.name, SUM(b.volume_cy) AS total_volume, c.countyname
        FROM counties_id c
        JOIN beaches_id b ON b.GEO_ID = c.GEO_ID
        WHERE b.year <= {b}
        GROUP BY c.GEO_ID, c.state, c.name, c.countyname
        '''
    df = pd.read_sql_query(myquery, con=engine)
    df.dropna(axis = 0, inplace = True)
    df['total_volume'] = df['total_volume'].astype(int)
    
    df_top10 = df.sort_values(by='total_volume', ascending=False).head(10)
    
    # Create a separate column for formatted total_volume
    df['f_total_volume'] = df['total_volume'].apply(lambda x: f"{x:,}")
    
    df_top10['county_state'] = df_top10['countyname'] + ', ' + df_top10['state']
    
    # Create a bar chart
    fig = px.bar(df_top10, x='county_state', y='total_volume',
                 labels={'county_state': 'County/State', 'total_volume': 'Total Volume'},
                 color = 'total_volume',
                 color_continuous_scale = px.colors.sequential.Mint
                )
    
    # Customize hover data
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>Total Volume: %{y:,}"
    )
    
   
    fig.update_layout(
        plot_bgcolor='white', 
        paper_bgcolor='white',  
        showlegend=False,
        xaxis=dict(showgrid=False),  
        yaxis=dict(showgrid=False),
    )
    
    return [fig]

@app.callback([Output(component_id = 'reefs', component_property = 'figure')],
             [Input(component_id = 'year1', component_property = 'value')])

def reefs(b):
    if b is None:
        b = 2022
    myquery= f'''
    SELECT *
    FROM reefs
    WHERE reefyear <= {b}
    '''
    df =gpd.GeoDataFrame.from_postgis(myquery, con = engine, geom_col='geometry')
    fig = px.scatter_mapbox(df, 
                        lat=df.geometry.y,
                        lon=df.geometry.x,
                        color='waterdepthfeet',
                        color_continuous_scale="Teal",
                        mapbox_style="carto-positron",
                        hover_name='reefname',
                        labels={'reefname': 'Reef Name',
                                'waterdepthfeet': 'Depth'}
                        )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return [fig]
    
if __name__ == "__main__":
    app.run_server(host="0.0.0.0", port=8050, debug=True)